In [1]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

/Users/ktg/miniforge3/envs/fastapi/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

text = "과거를 떠올려보자. 방송을 보던 우리의 모습을. 독보적인 매체는 TV였다. 온 가족이 둘러앉아 TV를 봤다. 간혹 가족들끼리 뉴스와 드라마, 예능 프로그램을 둘러싸고 리모컨 쟁탈전이 벌어지기도  했다. 각자 선호하는 프로그램을 ‘본방’으로 보기 위한 싸움이었다. TV가 한 대인지 두 대인지 여부도 그래서 중요했다. 지금은 어떤가. ‘안방극장’이라는 말은 옛말이 됐다. TV가 없는 집도 많다. 미디어의 혜 택을 누릴 수 있는 방법은 늘어났다. 각자의 방에서 각자의 휴대폰으로, 노트북으로, 태블릿으로 콘텐츠 를 즐긴다."

raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]))
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


'TV가 없는 집도 많아지고 미디어의 혜 택을 누릴 수 있는 방법은 늘어났다.'

In [33]:
raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
input_ids = torch.tensor([input_ids])

summary_text_ids = model.generate(
    input_ids=input_ids,
    bos_token_id=model.config.bos_token_id,
    eos_token_id=model.config.eos_token_id,
    length_penalty=1.0, # 길이에 대한 penalty값. 1보다 작은 경우 더 짧은 문장을 생성하도록 유도하며, 1보다 클 경우 길이가 더 긴 문장을 유도
    max_length=128,     # 요약문의 최대 길이 설정
    min_length=32,      # 요약문의 최소 길이 설정
    num_beams=4,        # 문장 생성시 다음 단어를 탐색하는 영역의 개수 
)

print(tokenizer.decode(summary_text_ids[0], skip_special_tokens=True))

TV가 없는 집도 많아지고 미디어의 혜 택을 누릴 수 있는 방법은 늘어났으며 각자의 방에서 각자의 휴대폰으로, 노트북으로, 태블릿으로 콘텐츠 를 즐긴다.


In [ ]:
import torch
print(f"""Pytorch Version: {torch.__version__}
torch.cuda.is_available: {torch.cuda.is_available()}
torch.backends.mps.is_built: {torch.backends.mps.is_built()}
torch.backends.mps.is_available: {torch.backends.mps.is_available()}
""")

In [8]:
print(torch.__version__)
print(torch.backends.mps.is_built())
print(torch.backends.mps.is_available())

device = "mps" if torch.backends.mps.is_available() else "cpu"
device = torch.device("mps")

device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device("cuda")
# device = torch.device("cpu")

print('torch.cuda.is_available: ', torch.cuda.is_available())
print('torch.backends.mps.is_available:', torch.backends.mps.is_available())

1.13.1
True
True
torch.cuda.is_available:  False
torch.backends.mps.is_available: True


In [12]:
import torch

print (f"PyTorch version:{torch.__version__}") # 1.12.1 이상
print(f"MPS 장치를 지원하도록 build 되었는지: {torch.backends.mps.is_built()}") # True 여야 합니다.
print(f"MPS 장치가 사용 가능한지: {torch.backends.mps.is_available()}") # True 여야 합니다.
!python -c 'import platform;print(platform.platform())'

PyTorch version:1.13.1
MPS 장치를 지원하도록 build 되었는지: True
MPS 장치가 사용 가능한지: True
macOS-13.0-arm64-arm-64bit
